Standard imports

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tweepy
import datetime
import json

Text cleaner imports

In [3]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
import string
from nltk.stem import PorterStemmer
import re

stop_words=stopwords.words('english')
punct=string.punctuation
stemmer=PorterStemmer()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ysgard\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


This imports the hedonometer words. Could be used for sentiment analysis.

In [178]:
hedonometer = pd.read_csv("../Hedonometer.csv")

#initialize the categorization
negative_words, neutral_words, positive_words, all_words = {}, {}, {}, {}

# how clearly valent do we want the words?
tolerance = 0

# demark words that are clearly negative or positive (neutral is within tolerance standard deviations of 5)
for i in hedonometer.index:
    score = (hedonometer['Happiness Score'][i]-5)/5
    if score+tolerance*hedonometer['Standard Deviation of Ratings'][i]/5<0:
        negative_words[hedonometer.Word[i]] = score
    elif score-tolerance*hedonometer['Standard Deviation of Ratings'][i]/5>5:
        positive_words[hedonometer.Word[i]] = score
    else: 
        neutral_words[hedonometer.Word[i]] = score
    all_words[hedonometer.Word[i]] = score

This can save stuff for later usage, in case they're handy.

In [30]:
def tweet_saver(tweets, file='TWagner.csv'):
    with open(file, 'a', encoding='utf8') as saveTweets:
        for tweet in tweets: 
            saveTweets.write(str(tweet._json)+'\n')
    
def user_saver(user, file='UUagner.csv'):
    with open(file, 'a', encoding='utf8') as saveUser:
        saveUser.write(str(user)+'\n')

Personal info please don't use.

In [10]:
ck = 'QxPH3Ud4nFeHxfiOsnzNr2sql'
cs = ''
at = '1393253063528431621-Tk8YAgtU3B4SwJa4DmHlhfGlcaiWN6'
ats = ''

This creates an instance of the API so that I can pull Tweets from Twitter.

In [11]:
# create OAuthHandler object
authenticator = tweepy.OAuthHandler(ck, cs)
# set access token and secret
authenticator.set_access_token(at, ats)
# create tweepy API object to fetch tweets
API = tweepy.API(authenticator, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

Practicing/experimenting with followers.

In [203]:
API.show_friendship(source_screen_name='DougEpisodes', target_id=1015024528337129473)

(Friendship(_api=<tweepy.api.API object at 0x000001F975D9C588>, _json={'id': 2485562569, 'id_str': '2485562569', 'screen_name': 'DougEpisodes', 'following': False, 'followed_by': True, 'live_following': False, 'following_received': None, 'following_requested': None, 'notifications_enabled': None, 'can_dm': True, 'blocking': None, 'blocked_by': None, 'muting': None, 'want_retweets': None, 'all_replies': None, 'marked_spam': None}, id=2485562569, id_str='2485562569', screen_name='DougEpisodes', following=False, followed_by=True, live_following=False, following_received=None, following_requested=None, notifications_enabled=None, can_dm=True, blocking=None, blocked_by=None, muting=None, want_retweets=None, all_replies=None, marked_spam=None),
 Friendship(_api=<tweepy.api.API object at 0x000001F975D9C588>, _json={'id': 1015024528337129473, 'id_str': '1015024528337129473', 'screen_name': 'newrockytweets', 'following': True, 'followed_by': False, 'following_received': None, 'following_request

Let's make a dataframe of activity

In [5]:
def get_activity(user, count=20, local=False):
    # Make sure we're finding the right user
    if local:
        with open(user, 'r', encoding='utf8') as reader:
            pass
    elif type(user)==str:
        apitivity = API.user_timeline(screen_name=user, count=count)
    elif type(user)==int:
        apitivity = API.user_timeline(user_id=user, count=count)
    else:
        raise Exception("Bad user.") 
        
    # Make a list of the statuses
    activiray = []
    #and add the relevant data
    for tweet in apitivity:
        try: 
            tweet.retweeted_status
            is_retweet = True
        except: 
            is_retweet = False
        activiray.append([tweet.text,
                         tweet.id,
                         is_retweet,
                         tweet.is_quote_status,
                         type(tweet.in_reply_to_status_id)==int,
                         tweet.retweet_count,
                         tweet.favorite_count,
                         tweet.created_at])
        
    # turn it into a dataframe
    activi pd.DataFrame(activiray, columns=['Text', 'Id', 'Is_retweet', 'Is_quote', 'Is_reply', 'Retweets', 'Favorites', 'Created'])

In [6]:
def prep_activity(activity, count=20, 
                   streak_lengths=[datetime.timedelta(minutes=30),datetime.timedelta(hours=6)], 
                   per_periods=[datetime.timedelta(hours=1),datetime.timedelta(days=1)]):
    # Did we already get the activity?
    if type(activity)!=pd.core.frame.DataFrame:
        activity = get_activity(activity, count=count)
    
    #clean the text
    cleets = []
    for twittre in activity.Text:
        #this removes mentions
        twittre = re.sub(r'@[A-Za-z0-9_]+', '', twittre)
        #this removes urls
        twittre = re.sub(r'https?:\/\/[A-Za-z0-9\.\/]+', '', twittre)        
        #this removes everything but letters from the str
        twittre = re.sub('[^a-zA-Z]',' ',twittre)
        #this makes everything lowercase, then turns everything that is separated by a space into its own str
        twittre = twittre.lower().split()
        #this removes stop words and stems everything else
        twittre = [stemmer.stem(word) for word in twittre if (word not in stop_words)]
        # this puts it all back together with spaces inbetween
        twittre = ' '.join(twittre)
        cleets.append(twittre)
    activity['Cleaned'] = cleets
    
    # record the time since the last status update
    tsl = [activity.Created[n].to_pydatetime() - activity.Created[n+1].to_pydatetime()
                                   for n in range(len(activity)-1)] + [np.nan]
    activity['Time_since_last'] = tsl.copy()
    
    #record how many times in a row the last activity was less than length 
    for length in streak_lengths:
        #record when it was less than length
        streak_break = [time>length for time in tsl[:-1]]+[True]
        
        #intialize beginning of a streak
        start = 0
        streak = []
        for end in range(len(streak_break)):
            #if this is the end of a streak,  
            if streak_break[end]:
                #record it,
                streak += [end+1-start for indx in range(end+1-start)]
                #and begin the next
                start = end+1
        
        #finally, add it to the data frame
        activity['Streak of '+str(length)] = streak
        
    #record how much activity occured within a given period 
    for period in per_periods:
        per = []
        for n in range(len(tsl)):
            elapsed = tsl[n]
            count, m = 0, 1
            #first we look at earlier tweets
            while (m+n<len(tsl)) and (elapsed<=period/2):
                count += 1
                try: elapsed += tsl[m+n]
                except: pass
                m += 1
            m = 0
            elapsed = datetime.timedelta(0)
            #then later tweets
            while n>=m and elapsed<=period/2:
                count += 1
                m += 1
                try: elapsed += tsl[n-m]
                except: pass
            per += [count]
        activity['Per '+str(period)] = per

In [8]:
with open('./erdos_core/0.txt', 'r', encoding='utf8') as reader:
    tweets = reader.readlines()

In [282]:
activity = get_activity('erdosinstitute')
prep_activity(activity)
activity

,Text,Id,Is_retweet,Is_quote,Is_reply,Reply_of,Replying_to,Retweets,Favorites,Created,Cleaned,Time_since_last,Streak of 1:00:00,"Streak of 1 day, 0:00:00",Per 1:00:00,"Per 1 day, 0:00:00"
0,RT @AstronomerPat: @RacineSwick @WorkWithVari ...,1395210702869118982,True,False,False,NaN,NaN,1,0,2021-05-20 02:52:07,rt one week data scienc lectur,2 days 12:09:26,1,1,1,1
1,Dreams can come true @AstronomerPat https://t....,1394302356267880449,False,True,False,NaN,NaN,0,1,2021-05-17 14:42:41,dream come true,0 days 00:00:38,2,2,2,2
2,@AstronomerPat @WorkWithVari Same excellent co...,1394302197006053379,False,False,True,1.394291e+18,9.130586e+17,0,1,2021-05-17 14:42:03,excel content screen,10 days 14:15:05,2,2,2,2
3,Our May Data Science Boot Camp is in full swin...,1390463130627674113,False,True,False,NaN,NaN,0,4,2021-05-07 00:26:58,may data scienc boot camp full swing photo credit,0 days 00:09:20,2,2,2,2
4,@AstronomerPat @MattOsborne71: This is the mos...,1390460782647906306,False,False,True,1.390296e+18,9.130586e+17,1,6,2021-05-07 00:17:38,retweet ever photo data scienc boot camp,6 days 12:42:47,2,2,2,2
5,RT @columbusbiz1st: How an OSU math professor ...,1388094491609997320,True,False,False,NaN,NaN,2,0,2021-04-30 11:34:51,rt osu math professor match ph privat sector job,1 days 10:16:08,1,1,1,1
6,Our May Data Science Boot Camp with over 350 g...,1387577050735812608,False,False,False,NaN,NaN,1,9,2021-04-29 01:18:43,may data scienc boot camp grad student postdoc...,6 days 09:23:46,1,1,1,1
7,Invitations to Industry continues tonight with...,1385260846515773444,False,False,False,NaN,NaN,1,1,2021-04-22 15:54:57,invit industri continu tonight kenneth maynard...,1 days 16:01:56,1,1,1,1
8,RT @OSUmath: The Hidden Figures course develop...,1384656382406840329,True,False,False,NaN,NaN,3,0,2021-04-20 23:53:01,rt hidden figur cours develop dr ranthoni edmo...,9 days 03:36:50,1,1,1,1
9,Invitations to Industry continues Monday 04/12...,1381340320051384321,False,False,False,NaN,NaN,0,0,2021-04-11 20:16:11,invit industri continu monday thoma sznigir st...,8 days 19:50:50,1,1,1,1


In [208]:
clean_activity(activity)

In [209]:
activity

,Text,Id,Created,Is reply,Reply_of,Replying_to,Retweets,Favorites,Is_retweet,Time_since_last,Streak of 1:00:00,"Per 1 day, 0:00:00"
0,RT @AstronomerPat: @RacineSwick @WorkWithVari ...,1395210702869118982,2021-05-20 02:52:07,False,NaN,NaN,1,0,True,2 days 12:09:26,1,1
1,Dreams can come true @AstronomerPat https://t....,1394302356267880449,2021-05-17 14:42:41,False,NaN,NaN,0,1,False,0 days 00:00:38,2,2
2,@AstronomerPat @WorkWithVari Same excellent co...,1394302197006053379,2021-05-17 14:42:03,True,1.394291e+18,9.130586e+17,0,1,False,10 days 14:15:05,2,2
3,Our May Data Science Boot Camp is in full swin...,1390463130627674113,2021-05-07 00:26:58,False,NaN,NaN,0,4,False,0 days 00:09:20,2,2
4,@AstronomerPat @MattOsborne71: This is the mos...,1390460782647906306,2021-05-07 00:17:38,True,1.390296e+18,9.130586e+17,1,6,False,6 days 12:42:47,2,2
5,RT @columbusbiz1st: How an OSU math professor ...,1388094491609997320,2021-04-30 11:34:51,False,NaN,NaN,2,0,True,1 days 10:16:08,1,1
6,Our May Data Science Boot Camp with over 350 g...,1387577050735812608,2021-04-29 01:18:43,False,NaN,NaN,1,9,False,6 days 09:23:46,1,1
7,Invitations to Industry continues tonight with...,1385260846515773444,2021-04-22 15:54:57,False,NaN,NaN,1,1,False,1 days 16:01:56,1,1
8,RT @OSUmath: The Hidden Figures course develop...,1384656382406840329,2021-04-20 23:53:01,False,NaN,NaN,3,0,True,9 days 03:36:50,1,1
9,Invitations to Industry continues Monday 04/12...,1381340320051384321,2021-04-11 20:16:11,False,NaN,NaN,0,0,False,8 days 19:50:50,1,1


Building a network

In [5]:
def build_core(user, breadth=10, depth=2, core=[], save=False):
    #type user as a screen name so this works
    howrite = 'a'
    
    # Make sure we're finding the right user
    if type(user)==str:
        user = API.get_user(screen_name=user).id
        howrite = 'w'
    elif type(user)!=int:
        raise Exception("Bad user.")
    
    #make sure we have good data
    try:
        friends = API.friends_ids(user_id=user)
        friends[0]
    except: 
        return False
    
    #one list for user's mutuals, one list for everyone
    mutuals = []
    core.append(user)
    if save:
        try:
            with open(save, howrite) as save_file:
                save_file.write(str(user)+'\n')
        except: 
            raise Exception('Bad save file')
    
    #we won't delve too deep or too greedily
    if depth:
        #find random friends of user who follow user back (mutuals)
        for friend in np.random.permutation(friends):
            friend = int(friend)
            if friend not in core:
                if API.show_friendship(source_id=friend, target_id=user)[0].following:
                    #find their mutuals, but only add them to the list if it's good data
                    if build_core(friend, breadth=breadth, depth=depth-1, core=core, save=save):
                        #add them to the list
                        mutuals.append(friend)
                    #but only some of them
                    if len(mutuals)==breadth:
                        print(friend,'has mutuals',mutuals)
                        break
    
    #and here's the list
    return core

In [6]:
def build_crust(core, breadth=100, crust=[], save=False):
    #initialize a list for the crust
    crust = core.copy()
    
    #and a save file if neccessary
    if save:
        try:
            with open(save, 'w') as save_file:
                for core_member in core:
                    save_file.write(str(core_member)+'\n')
        except: 
            raise Exception('Bad save file')
        
    #for each member of the core...
    for core_member in core.copy():   
        skip_number = 0
        #add breadth number...
        follow_number = 0
        #of the core  member's...
        followers = np.random.permutation(API.followers_ids(user_id=core_member))
        
        #followers to the crust...
        for follower in followers:
            follower = int(follower)
            
            #if they are accessible...
            try:
                getted = API.get_user(user_id=follower)
                if getted.protected:
                    raise Exception('Skip this one')
            except:
                skip_number += 1
                continue
                
            #and aren't already in it
            if follower not in crust:
                crust.append(follower)
                if save:
                    with open(save, 'a') as save_file:
                        save_file.write(str(follower)+'\n')
                follow_number += 1
                if follow_number == breadth:
                    break
            else:
                skip_number += 1
        #keep tabs on how the script is going
        print('got',follow_number,'followers of',core_member,'while skipping',skip_number)
    
    return crust

In [53]:
def frame_data(core, crust, data=[], save=False, restart=0):
    #we assign core members as features
    columns = ['user']+['core_'+str(n) for n in range(len(core))]
    if save and not restart: 
        with open(save, 'r') as save_file:
            text = [''] + save_file.readlines()
        for column in columns:
            text[0] += ',' + str(column)
        text[0] += '\n'
        with open(save, 'w+') as save_file:
            save_file.writelines(text)
        
    #now we see which crust members are following which core members
    for n, crust_member in [x for x in enumerate(crust) if x[0]>=restart]:
        #this will hold what we find
        next_line = [crust_member]
        #look up who the crust member is following
        following, pages = API.friends_ids(user_id=crust_member, cursor=-1)
        
        for core_member in core:
            #page through if necessary
            while pages[1] and (core_member not in following):
                more_following, pages = API.friends_ids(user_id=crust_member, cursor=pages[1])
                following += more_following
            next_line.append(core_member in following)
        
        #We'll say people follow themselves
        if n < len(core):
            next_line[n]==True
        
        #now we store it
        data.append(next_line)
        if save: 
            with open(save, 'r') as save_file:
                text = save_file.readlines()
            text[n+1] = text[n+1][:-1]
            for datum in next_line:
                text[n+1] += ',' + str(datum)
            text[n+1] += '\n'
            with open(save, 'w') as save_file:
                save_file.writelines(text)
        print('Stored data for member', n+1, 'of', len(crust))
    
    #and finally put it all together
    return pd.DataFrame(data, index=crust, columns=columns)

In [6]:
tomi_core = build_core('TomiLahren', save='Tomi_core.txt')

1052051030 has mutuals [599767159, 1003398199, 707246777515245572, 277158689, 27104113, 18699142, 1631671458, 44691739, 4235197337, 1052051030]


Rate limit reached. Sleeping for: 890


716326749823438848 has mutuals [180705418, 1066585043626311680, 3354214695, 1099695707341180928, 1161814802, 96698038, 762470063261704192, 419084104, 1269613348003471360, 716326749823438848]


Rate limit reached. Sleeping for: 887


1470633924 has mutuals [262834701, 236287355, 36837114, 462494796, 48761924, 367479753, 395620849, 2740563252, 32691725, 1470633924]


...

16175706 has mutuals [2483104627, 58807922, 842327754301423616, 2436338828, 465092949, 4802754257, 856877583803265025, 762789921933307904, 618356934, 16175706]


Rate limit reached. Sleeping for: 888


2665748041 has mutuals [821557387, 19613907, 756198843042193408, 18774951, 267944170, 3310505366, 37718165, 499771067, 45660732, 2665748041]
111660650 has mutuals [2149677560, 2730264319, 902200087, 217634501, 3165740402, 18410315, 449850814, 434458299, 3407890707, 111660650]


Rate limit reached. Sleeping for: 883


30721022 has mutuals [1584092041, 142297864, 72002979, 540240770, 34951162, 280339608, 2272305950, 2268158728, 24880132, 30721022]
39797085 has mutuals [36432879, 18080108, 56319954, 107125748, 98240178, 16869718, 71675903, 25443684, 4532518512, 39797085]


In [ ]:

for i, crusty in enumerate(crust):
    tweets = API.user_timeline(user_id=crusty, count=100)
    tweets_array.append(tweets)
    tweet_saver(tweets, '../Messing around/erdos_crust/'+str(i)+'.txt')
    print('Found',len(tweets),'from number', i)

Found 98 from number 0
Found 100 from number 1
Found 99 from number 2
Found 61 from number 3
Found 64 from number 4
Found 100 from number 5
Found 6 from number 6
Found 100 from number 7
Found 8 from number 8
Found 100 from number 9
Found 3 from number 10
Found 0 from number 11
Found 99 from number 12
Found 100 from number 13
Found 100 from number 14
Found 100 from number 15
Found 100 from number 16
Found 99 from number 17
Found 100 from number 18
Found 42 from number 19
Found 100 from number 20
Found 100 from number 21
Found 100 from number 22
Found 36 from number 23
Found 100 from number 24
Found 100 from number 25
Found 100 from number 26
Found 63 from number 27
Found 100 from number 28
Found 100 from number 29
Found 0 from number 30
Found 99 from number 31
Found 99 from number 32
Found 98 from number 33
Found 100 from number 34
Found 100 from number 35
Found 16 from number 36
Found 2 from number 37
Found 68 from number 38
Found 99 from number 39
Found 100 from number 40
Found 100 fr

Found 100 from number 327
Found 2 from number 328
Found 100 from number 329
Found 54 from number 330
Found 100 from number 331
Found 100 from number 332
Found 100 from number 333
Found 100 from number 334
Found 16 from number 335
Found 100 from number 336
Found 8 from number 337
Found 97 from number 338
Found 7 from number 339
Found 100 from number 340
Found 100 from number 341
Found 99 from number 342
Found 100 from number 343
Found 97 from number 344
Found 100 from number 345
Found 100 from number 346
Found 100 from number 347
Found 100 from number 348
Found 100 from number 349
Found 100 from number 350
Found 100 from number 351
Found 100 from number 352
Found 1 from number 353
Found 100 from number 354
Found 100 from number 355
Found 100 from number 356
Found 36 from number 357
Found 100 from number 358
Found 100 from number 359
Found 100 from number 360
Found 1 from number 361
Found 100 from number 362
Found 100 from number 363
Found 100 from number 364
Found 100 from number 365
Fo

Found 98 from number 650
Found 100 from number 651
Found 100 from number 652
Found 99 from number 653
Found 100 from number 654
Found 99 from number 655
Found 100 from number 656
Found 100 from number 657
Found 100 from number 658
Found 100 from number 659
Found 100 from number 660
Found 100 from number 661
Found 100 from number 662
Found 100 from number 663
Found 100 from number 664
Found 81 from number 665
Found 99 from number 666
Found 100 from number 667
Found 100 from number 668
Found 100 from number 669
Found 100 from number 670
Found 100 from number 671
Found 7 from number 672
Found 100 from number 673
Found 20 from number 674
Found 100 from number 675
Found 0 from number 676
Found 100 from number 677
Found 100 from number 678
Found 100 from number 679
Found 97 from number 680
Found 100 from number 681
Found 100 from number 682
Found 100 from number 683
Found 100 from number 684
Found 100 from number 685
Found 100 from number 686
Found 100 from number 687
Found 100 from number 6

In [5]:
2

2

In [75]:
with open('erdos_full_sample.csv', 'r') as reader:
    lines = reader.readlines()

In [81]:
crust += [int(line) for line in lines[154:]]

In [66]:
API.get_user(screen_name='UnprotectedMe')

User(_api=<tweepy.api.API object at 0x000002327876BF48>, _json={'id': 80076948, 'id_str': '80076948', 'name': 'khadija', 'screen_name': 'unprotectedme', 'location': '', 'profile_location': None, 'description': '', 'url': None, 'entities': {'description': {'urls': []}}, 'protected': True, 'followers_count': 2, 'friends_count': 63, 'listed_count': 0, 'created_at': 'Mon Oct 05 17:24:55 +0000 2009', 'favourites_count': 0, 'utc_offset': None, 'time_zone': None, 'geo_enabled': False, 'verified': False, 'statuses_count': 5, 'lang': None, 'contributors_enabled': False, 'is_translator': False, 'is_translation_enabled': False, 'profile_background_color': '1A1B1F', 'profile_background_image_url': 'http://abs.twimg.com/images/themes/theme9/bg.gif', 'profile_background_image_url_https': 'https://abs.twimg.com/images/themes/theme9/bg.gif', 'profile_background_tile': False, 'profile_image_url': 'http://abs.twimg.com/sticky/default_profile_images/default_profile_normal.png', 'profile_image_url_https':

Exploring activity/negativity relationship

In [16]:
from joblib import load

vectord = load('../vectord.joblib') 
model = load('../model.joblib') 

#tweets must be cleaned first
def sentiment(tweets, analogue=False):
    if analogue: 
        probs = model.predict_proba(vectord.transform(tweets).toarray())
        return probs[:,1]-probs[:,0]
    else: return model.predict(vectord.transform(tweets).toarray())

In [17]:
erdos_core = list(pd.read_csv('erdos_full_core.csv')['user'])

In [19]:
erdos_activities = pd.DataFrame()
for n, user in enumerate(erdos_core):
    try:
        activity = get_activity(user, count=1000)
        prep_activity(activity)
        activity['Predicted_polarity'] = sentiment(activity.Cleaned, True)
        activity['User'] = [user for n in range(len(activity))]
        erdos_activities = pd.concat([erdos_activities, activity])
    except: continue
    if not n%10: print(n)


0
10
20
30
40
50
60
70
80
90
100


In [20]:
erdos_activities[['Is_retweet','Is_quote','Is_reply','Retweets','Favorites','Created','Time_since_last','Streak of 0:30:00','Streak of 6:00:00','Per 1:00:00',"Per 1 day, 0:00:00",'Predicted_polarity','User']].to_csv('Erdos_core_tweets.csv', index=False)

Exploring followship/negativity relationship

,Text,Id,Is_retweet,Is_quote,Is_reply,Retweets,Favorites,Created,Cleaned,Time_since_last,Streak of 0:30:00,Streak of 6:00:00,Per 1:00:00,"Per 1 day, 0:00:00",Predicted_polarity,User
0,RT @AstronomerPat: @RacineSwick @WorkWithVari ...,1395210702869118982,True,False,False,1,0,2021-05-20 02:52:07,rt one week data scienc lectur,2 days 12:09:26,1,1,1,1,-0.026356,840727059001417728
1,Dreams can come true @AstronomerPat https://t....,1394302356267880449,False,True,False,0,1,2021-05-17 14:42:41,dream come true,0 days 00:00:38,2,2,2,2,-0.023730,840727059001417728
2,@AstronomerPat @WorkWithVari Same excellent co...,1394302197006053379,False,False,True,0,1,2021-05-17 14:42:03,excel content screen,10 days 14:15:05,2,2,2,2,0.911806,840727059001417728
3,Our May Data Science Boot Camp is in full swin...,1390463130627674113,False,True,False,0,4,2021-05-07 00:26:58,may data scienc boot camp full swing photo credit,0 days 00:09:20,2,2,2,2,-0.026356,840727059001417728
4,@AstronomerPat @MattOsborne71: This is the mos...,1390460782647906306,False,False,True,1,6,2021-05-07 00:17:38,retweet ever photo data scienc boot camp,6 days 12:42:47,2,2,2,2,-0.026356,840727059001417728
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18651,@AltonStandifer saw this. looking forward to t...,1312421430915665925,False,False,True,0,0,2020-10-03 15:57:07,saw look forward talk also thought iron ask wa...,0 days 00:00:14,2,2,2,2,0.010908,556041652
18652,Many thanks to @karinbrulliard for including h...,1312421371130130442,False,False,False,1,3,2020-10-03 15:56:53,mani thank includ interview towngown stori,0 days 18:09:22,2,2,2,2,0.725753,556041652
18653,@Jeff_Piestrak @PezeshkiCharles @ugobardi In t...,1312147224558665728,False,False,True,0,2,2020-10-02 21:47:31,famili scienc field gener substanti evid regar,0 days 03:59:29,1,3,1,3,0.188250,556041652
18654,@Jeff_Piestrak @PezeshkiCharles @ugobardi I've...,1312086958248779777,False,False,True,0,2,2020-10-02 17:48:02,like great deal embri work past line seem espec,0 days 01:09:22,1,3,1,3,0.212042,556041652


In [5]:
tomi_activities = pd.read_csv('Tomi_core_tweets.csv')
with open('Tomi_core.txt', 'r') as reader:
    tomi_core = [int(user) for user in reader.readlines()]

averages = []
for user in tomi_core:
    polarities = tomi_activities.Predicted_polarity[[i for i in tomi_activities.index if tomi_activities.User[i]==user]]
    mean = (0 if len(polarities)==0 else np.mean(polarities))
    averages.append(mean)

In [44]:
averages

[0.04114194694522725,
 0.07433786263837878,
 0.16074189954019405,
 0.19532417971510796,
 -0.005821568789379526,
 0.3589881429514164,
 0.0303263620883977,
 0.06241997161555755,
 0.11038947990641014,
 0.22877099623412356,
 0.1672776981800818,
 -0.03925673271201909,
 -0.023961598843165486,
 -0.08186073376243334,
 0.006291437858897314,
 -0.09945117382384706,
 -0.019014694255329482,
 -0.028951450990162254,
 -0.029918858552976588,
 -0.061489312511163484,
 -0.042581423800928296,
 -0.022716581244002935,
 -0.05278408387109866,
 -0.053669453525394345,
 -0.008839471434550949,
 -0.08402331021794951,
 0.07790118762774641,
 0.0023395298476488026,
 0.050403777920654085,
 -0.07684738327803117,
 0.12862584984475964,
 -0.0033051517951969817,
 0.01658910791156576,
 0.01997550259432464,
 0.08591942425089485,
 -0.0830932865396215,
 0.06783290645299621,
 0.06502560068565374,
 -0.05011216896483873,
 0.03351758251158142,
 0.08840775289229379,
 -0.03686137221410989,
 0.020718566784352868,
 -0.01392467812966254

In [7]:
tomi_cross_core = pd.read_csv('Tomi_cross_core.csv')
tomi_cross_core['Average_polarity'] = averages
tomi_cross_core.to_csv('Tomi_cross_core.csv', index=False)

User(_api=<tweepy.api.API object at 0x000001AACC353BC8>, _json={'id': 2665748041, 'id_str': '2665748041', 'name': 'Rosanna Landis Weaver', 'screen_name': 'LandisWeaver', 'location': '', 'profile_location': None, 'description': 'I write reports on overpaid CEOs & how funds vote on pay. I read everything I can. I tweet social justice, corporate governance, books, gardens, adoption, kids', 'url': 'https://t.co/TbT3jqonCd', 'entities': {'url': {'urls': [{'url': 'https://t.co/TbT3jqonCd', 'expanded_url': 'https://www.asyousow.org/our-work/ceo-pay', 'display_url': 'asyousow.org/our-work/ceo-p…', 'indices': [0, 23]}]}, 'description': {'urls': []}}, 'protected': False, 'followers_count': 5310, 'friends_count': 4012, 'listed_count': 99, 'created_at': 'Mon Jul 21 12:05:53 +0000 2014', 'favourites_count': 126820, 'utc_offset': None, 'time_zone': None, 'geo_enabled': False, 'verified': False, 'statuses_count': 36177, 'lang': None, 'status': {'created_at': 'Fri May 28 18:00:44 +0000 2021', 'id': 13